In [1]:


import numpy as np # linear algebra
import pandas as pd
import plotly.express as px
from IPython.display import VimeoVideo
from scipy.stats.mstats import trimmed_var
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.utils.validation import check_is_fitted


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



/kaggle/input/data-dictionary-for-customer-segmentation-usa/hcbk.htm
/kaggle/input/survey-of-consumer-finances-data/SCFP2019.csv


`Prepare Data`

`a.Import`

Task 6.3.1: Rewrite your wrangle function from the last lesson so that it returns a DataFrame of households whose net worth is less than $2 million and that have been turned down for credit or feared being denied credit in the past 5 years (see "TURNFEAR").

In [2]:
def wrangle(filepath):
    # Read file into DataFrame
    df = pd.read_csv(filepath)
    mask = (df["TURNFEAR"] == 1) & (df["NETWORTH"] < 2e6)
    df = df[mask]
    return df

In [3]:
df = wrangle("/kaggle/input/survey-of-consumer-finances-data/SCFP2019.csv")

print("df type:", type(df))
print("df shape:", df.shape)
df.head()

df type: <class 'pandas.core.frame.DataFrame'>
df shape: (4407, 356)


,YY1,Y1,WGT,HHSEX,AGE,AGECL,EDUC,EDCL,MARRIED,KIDS,...,NWCAT,INCCAT,ASSETCAT,NINCCAT,NINC2CAT,NWPCTLECAT,INCPCTLECAT,NINCPCTLECAT,INCQRTCAT,NINCQRTCAT
5,2,21,3790.476607,1,50,3,8,2,1,3,...,1,2,1,2,1,1,4,4,2,2
6,2,22,3798.868505,1,50,3,8,2,1,3,...,1,2,1,2,1,1,4,3,2,2
7,2,23,3799.468393,1,50,3,8,2,1,3,...,1,2,1,2,1,1,4,4,2,2
8,2,24,3788.076005,1,50,3,8,2,1,3,...,1,2,1,2,1,1,4,4,2,2
9,2,25,3793.066589,1,50,3,8,2,1,3,...,1,2,1,2,1,1,4,4,2,2


`b. Explore`

Task 6.3.2: Calculate the variance for all the features in df, and create a Series top_ten_var with the 10 features with the largest variance.

In [4]:
x = df["DEBT"]
x.head()

5    14142.304636
6    14605.986755
7    17735.841060
8    16344.794702
9    17851.761589
Name: DEBT, dtype: float64

In [5]:
x.var()

24575632656.518795

In [6]:
df.var().sort_values().tail(10)

PLOAN1      1.508048e+10
ACTBUS      1.677175e+10
BUS         1.683586e+10
KGTOTAL     1.801415e+10
DEBT        2.457563e+10
HOUSES      3.025930e+10
NHNFIN      3.958070e+10
NETWORTH    5.446911e+10
NFIN        7.370790e+10
ASSET       9.864096e+10
dtype: float64

In [7]:
# Calculate variance, get 10 largest features
top_ten_var = df.var().sort_values().tail(10)

print("top_ten_var type:", type(top_ten_var))
print("top_ten_var shape:", top_ten_var.shape)
top_ten_var

top_ten_var type: <class 'pandas.core.series.Series'>
top_ten_var shape: (10,)


PLOAN1      1.508048e+10
ACTBUS      1.677175e+10
BUS         1.683586e+10
KGTOTAL     1.801415e+10
DEBT        2.457563e+10
HOUSES      3.025930e+10
NHNFIN      3.958070e+10
NETWORTH    5.446911e+10
NFIN        7.370790e+10
ASSET       9.864096e+10
dtype: float64

Task 6.3.3: Use plotly express to create a horizontal bar chart of top_ten_var. Be sure to label your x-axis "Variance", the y-axis "Feature", and use the title "SCF: High Variance Features".

In [8]:
# Create horizontal bar chart of `top_ten_var`
fig = px.bar(
    x=top_ten_var,
    y=top_ten_var.index,
    title="SCF: High Variance Features"
)
fig.update_layout(xaxis_title="Variance", yaxis_title="Feature")

fig.show()

One thing that we've seen throughout this project is that many of the wealth indicators are highly skewed, with a few outlier households having enormous wealth. Those outliers can affect our measure of variance. Let's see if that's the case with one of the features from top_five_var.

Task 6.3.4: Use plotly express to create a horizontal boxplot of "NHNFIN" to determine if the values are skewed. Be sure to label the x-axis "Value [$]", and use the title "Distribution of Non-home, Non-Financial Assets".

In [9]:
# Create a boxplot of `NHNFIN`
fig = px.box(
    data_frame=df,
    x="NHNFIN",
    title="Distribution of Non-home, Non-Financial Assets"
)
fig.update_layout(xaxis_title="Value ($)")

fig.show()

The dataset is massively right-skewed because of the huge outliers on the right side of the distribution. Even though we already excluded households with a high net worth with our wrangle function, the variance is still being distorted by some extreme outliers.

The best way to deal with this is to look at the trimmed variance, where we remove extreme values before calculating variance. We can do this using the trimmed_variance function from the SciPy library.

Task 6.3.5: Calculate the trimmed variance for the features in df. Your calculations should not include the top and bottom 10% of observations. Then create a Series top_ten_trim_var with the 10 features with the largest variance.

In [10]:
trimmed_var(df["DEBT"])

4099576134.443913

In [11]:
df.apply(trimmed_var, limits=(0.1, 0.1)).sort_values().tail(10)

WAGEINC     7.415894e+08
HOMEEQ      9.548834e+08
NH_MORT     1.755886e+09
MRTHEL      1.819848e+09
PLOAN1      1.902850e+09
NETWORTH    4.011771e+09
DEBT        4.099576e+09
HOUSES      6.508833e+09
NFIN        1.108097e+10
ASSET       1.530852e+10
dtype: float64

In [12]:
# Calculate trimmed variance
top_ten_trim_var = df.apply(trimmed_var, limits=(0.1, 0.1)).sort_values().tail(10)

print("top_ten_trim_var type:", type(top_ten_trim_var))
print("top_ten_trim_var shape:", top_ten_trim_var.shape)
top_ten_trim_var

top_ten_trim_var type: <class 'pandas.core.series.Series'>
top_ten_trim_var shape: (10,)


WAGEINC     7.415894e+08
HOMEEQ      9.548834e+08
NH_MORT     1.755886e+09
MRTHEL      1.819848e+09
PLOAN1      1.902850e+09
NETWORTH    4.011771e+09
DEBT        4.099576e+09
HOUSES      6.508833e+09
NFIN        1.108097e+10
ASSET       1.530852e+10
dtype: float64

Task 6.3.6: Use plotly express to create a horizontal bar chart of top_ten_trim_var. Be sure to label your x-axis "Trimmed Variance", the y-axis "Feature", and use the title "SCF: High Variance Features".

In [13]:
# Create horizontal bar chart of `top_ten_trim_var`
fig = px.bar(
    x=top_ten_trim_var,
    y=top_ten_trim_var.index,
    title="SCF: High Variance Features"
)
fig.update_layout(xaxis_title="Trimmed Variance", yaxis_title="Feature")

fig.show()

There are three things to notice in this plot. First, the variances have decreased a lot. In our previous chart, the x-axis went up to $80 billion; this one goes up to $15 billion. Second, the top 10 features have changed a bit. All the features relating to business ownership ("...BUS") are gone. Finally, we can see that there are big differences in variance from feature to feature. For example, the variance for "WAGEINC" is around than $500 million, while the variance for "ASSET" is nearly $12 billion. In other words, these features have completely different scales. This is something that we'll need to address before we can make good clusters.

Task 6.3.7: Generate a list high_var_cols with the column names of the five features with the highest trimmed variance.

In [14]:
top_ten_trim_var.tail(5).index.to_list()

['NETWORTH', 'DEBT', 'HOUSES', 'NFIN', 'ASSET']

In [15]:
high_var_cols = top_ten_trim_var.tail(5).index.to_list()

print("high_var_cols type:", type(high_var_cols))
print("high_var_cols len:", len(top_ten_trim_var))
high_var_cols

high_var_cols type: <class 'list'>
high_var_cols len: 10


['NETWORTH', 'DEBT', 'HOUSES', 'NFIN', 'ASSET']

`c. Split`

Now that we've gotten our data to a place where we can use it, we can follow the steps we've used before to build a model, starting with a feature matrix.

Task 6.3.8: Create the feature matrix X. It should contain the five columns in high_var_cols.

In [16]:
X = df[high_var_cols]

print("X type:", type(X))
print("X shape:", X.shape)
X.head()

X type: <class 'pandas.core.frame.DataFrame'>
X shape: (4407, 5)


,NETWORTH,DEBT,HOUSES,NFIN,ASSET
5,-7778.267550,14142.304636,0.0,4520.900662,6364.037086
6,-5459.856954,14605.986755,0.0,7302.993377,9146.129801
7,-9406.950993,17735.841060,0.0,6491.549669,8328.890066
8,-2909.605298,16344.794702,0.0,11592.052980,13435.189404
9,-6624.858278,17851.761589,0.0,9389.562914,11226.903311


`Build Model`

`Iterate`

we had a scale issue among our features. That issue can make it harder to cluster the data, so we'll need to fix that to help our analysis along. One strategy we can use is standardization, a statistical method for putting all the variables in a dataset on the same scale. Let's explore how that works here. Later, we'll incorporate it into our model pipeline.

Task 6.3.9: Create a DataFrame X_summary with the mean and standard deviation for all the features in X.

In [17]:
X.mean()

NETWORTH     83364.456280
DEBT         83742.853032
HOUSES       84402.639966
NFIN        133516.975569
ASSET       167107.309312
dtype: float64

In [18]:
X_summary = X.aggregate(["mean", "std"]).astype(int)

print("X_summary type:", type(X_summary))
print("X_summary shape:", X_summary.shape)
X_summary

X_summary type: <class 'pandas.core.frame.DataFrame'>
X_summary shape: (2, 5)


,NETWORTH,DEBT,HOUSES,NFIN,ASSET
mean,83364,83742,84402,133516,167107
std,233386,156766,173952,271491,314071


That's the information we need to standardize our data, so let's make it happen.

Task 6.3.10: Create a StandardScaler transformer, use it to transform the data in X, and then put the transformed data into a DataFrame named X_scaled.

In [19]:
x = X["DEBT"]
print("mean:", round(x.mean()))
print("std:", round(x.std()))

mean: 83743
std: 156766


In [20]:
x_scaled = (x - x.mean()) / x.std()
print("mean:", round(x_scaled.mean()))
print("std:", round(x_scaled.std()))

mean: 0
std: 1


In [21]:
# Instantiate transformer
ss = StandardScaler()

# Transform `X`
X_scaled_data = ss.fit_transform(X)

# Put `X_scaled_data` into DataFrame
X_scaled = pd.DataFrame(X_scaled_data, columns=X.columns)

print("X_scaled type:", type(X_scaled))
print("X_scaled shape:", X_scaled.shape)
X_scaled.head()

X_scaled type: <class 'pandas.core.frame.DataFrame'>
X_scaled shape: (4407, 5)


,NETWORTH,DEBT,HOUSES,NFIN,ASSET
0,-0.390568,-0.444027,-0.485262,-0.475192,-0.511863
1,-0.380633,-0.441069,-0.485262,-0.464943,-0.503004
2,-0.397547,-0.421102,-0.485262,-0.467932,-0.505606
3,-0.369704,-0.429976,-0.485262,-0.449143,-0.489346
4,-0.385625,-0.420362,-0.485262,-0.457257,-0.496378


As you can see, all five of the features use the same scale now. But just to make sure, let's take a look at their mean and standard deviation.

Task 6.3.11: Create a DataFrame X_scaled_summary with the mean and standard deviation for all the features in X_scaled.

In [22]:
X_scaled_summary = X_scaled.aggregate(["mean", "std"]).astype(int)

print("X_scaled_summary type:", type(X_scaled_summary))
print("X_scaled_summary shape:", X_scaled_summary.shape)
X_scaled_summary

X_scaled_summary type: <class 'pandas.core.frame.DataFrame'>
X_scaled_summary shape: (2, 5)


,NETWORTH,DEBT,HOUSES,NFIN,ASSET
mean,0,0,0,0,0
std,1,1,1,1,1


And that's what it should look like. Remember, standardization takes all the features and scales them so that they all have a mean of 0 and a standard deviation of 1.

Now that we can compare all our data on the same scale, we can start making clusters. Just like we did last time, we need to figure out how many clusters we should have.

Task 6.3.12: Use a for loop to build and train a K-Means model where n_clusters ranges from 2 to 12 (inclusive). Your model should include a StandardScaler. Each time a model is trained, calculate the inertia and add it to the list inertia_errors, then calculate the silhouette score and add it to the list silhouette_scores.

In [23]:
n_clusters = range(2, 13)
inertia_errors = []
silhouette_scores = []

# Add `for` loop to train model and calculate inertia, silhouette score.
for k in n_clusters:
    # Build model
    model = make_pipeline(
        StandardScaler(),
        KMeans(n_clusters=k, random_state=42),
    )
    
    # Train model
    model.fit(X)
    
    # Calculate intertia
    inertia_errors.append(model.named_steps["kmeans"].inertia_)
    
    # Calculate silhouette score
    silhouette_scores.append(
        silhouette_score(X, model.named_steps["kmeans"].labels_)
    )

# print("inertia_errors type:", type(inertia_errors))
# print("inertia_errors len:", len(inertia_errors))
print("Inertia:", inertia_errors)
print()
# print("silhouette_scores type:", type(silhouette_scores))
# print("silhouette_scores len:", len(silhouette_scores))
print("Silhouette Scores:", silhouette_scores)

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will chang

Inertia: [10900.236324338819, 7282.155740431512, 5940.004021846663, 5016.284489735257, 4300.181197638106, 3748.060047242459, 3158.784165522527, 2871.294771925613, 2645.073208496581, 2426.0901780320455, 2289.6272923707534]

Silhouette Scores: [0.7456405015578531, 0.7054175832593965, 0.6884218299913217, 0.6579331021067493, 0.6502914919387532, 0.6530226546586374, 0.6570872685116089, 0.6308001765433737, 0.5705499833730145, 0.5870514197590041, 0.5806437908580178]


Just like last time, let's create an elbow plot to see how many clusters we should use.

Task 6.3.13: Use plotly express to create a line plot that shows the values of inertia_errors as a function of n_clusters. Be sure to label your x-axis "Number of Clusters", your y-axis "Inertia", and use the title "K-Means Model: Inertia vs Number of Clusters".

In [24]:
# Create line plot of `inertia_errors` vs `n_clusters`
fig = px.line(
    x=n_clusters,
    y=inertia_errors, 
    title="K-Means Model: Inertia vs Number of Clusters"
)
fig.update_layout(xaxis_title="Number of Clusters", yaxis_title="Inertia")

fig.show()

You can see that the line starts to flatten out around 4 or 5 clusters.

Task 6.3.14: Use plotly express to create a line plot that shows the values of silhouette_scores as a function of n_clusters. Be sure to label your x-axis "Number of Clusters", your y-axis "Silhouette Score", and use the title "K-Means Model: Silhouette Score vs Number of Clusters".

In [25]:
# Create a line plot of `silhouette_scores` vs `n_clusters`
fig = px.line(
    x=n_clusters,
    y=silhouette_scores, 
    title="K-Means Model: Silhouette Score vs Number of Clusters"
)
fig.update_layout(xaxis_title="Number of Clusters", yaxis_title="Silhouette Score")

fig.show()

This one's a little less straightforward, but we can see that the best silhouette scores occur when there are 3 or 4 clusters.

Putting the information from this plot together with our inertia plot, it seems like the best setting for n_clusters will be 4.

Task 6.3.15: Build and train a new k-means model named final_model. Use the information you gained from the two plots above to set an appropriate value for the n_clusters argument. Once you've built and trained your model, submit it to the grader for evaluation.

In [26]:
# Build model
final_model = make_pipeline(
    StandardScaler(),
    KMeans(n_clusters=4, random_state=42)

)

# Fit model to data
final_model.fit(X)

# # Assert that model has been fit to data
# check_is_fitted(final_model)

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



Pipeline(steps=[('standardscaler', StandardScaler()),
                ('kmeans', KMeans(n_clusters=4, random_state=42))])

`Communicate`

Task 6.3.16: Extract the labels that your final_model created during training and assign them to the variable labels.

In [27]:
labels = final_model.named_steps["kmeans"].labels_

# print("labels type:", type(labels))
# print("labels len:", len(labels))
print(labels[:5])

[0 0 0 0 0]


Task 6.3.17: Create a DataFrame xgb that contains the mean values of the features in X for each of the clusters in your final_model.

In [28]:
xgb = X.groupby(labels).mean()

# print("xgb type:", type(xgb))
# print("xgb shape:", xgb.shape)
xgb

,NETWORTH,DEBT,HOUSES,NFIN,ASSET
0,14462.236824,28700.331814,14142.169056,2.952613e+04,4.316257e+04
1,151975.661877,255944.454527,285870.208697,3.556518e+05,4.079201e+05
2,829711.516225,842178.588076,901553.807943,1.511477e+06,1.671890e+06
3,832583.508518,150826.735106,297009.934498,6.967930e+05,9.834102e+05


Now that we have a DataFrame, let's make a bar chart and see how our clusters differ.

Task 6.3.18: Use plotly express to create a side-by-side bar chart from xgb that shows the mean of the features in X for each of the clusters in your final_model. Be sure to label the x-axis "Cluster", the y-axis "Value [$]", and use the title "Mean Household Finances by Cluster".

In [29]:
# Create side-by-side bar chart of `xgb`
fig = px.bar(
    xgb,
    barmode="group",
    title="Mean Household Finances by Cluster",
)
fig.update_layout(xaxis_title="Cluster", yaxis_title="Value [$]")

fig.show()

/opt/conda/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



Remember that our clusters are based partially on NETWORTH, which means that the households in the 0 cluster have the smallest net worth, and the households in the 2 cluster have the highest. Based on that, there are some interesting things to unpack here.

First, take a look at the DEBT variable. You might think that it would scale as net worth increases, but it doesn't. The lowest amount of debt is carried by the households in cluster 2, even though the value of their houses (shown in green) is roughly the same. You can't really tell from this data what's going on, but one possibility might be that the people in cluster 2 have enough money to pay down their debts, but not quite enough money to leverage what they have into additional debts. The people in cluster 3, by contrast, might not need to worry about carrying debt because their net worth is so high.

Finally, since we started out this project looking at home values, take a look at the relationship between DEBT and HOUSES. The value of the debt for the people in cluster 0 is higher than the value of their houses, suggesting that most of the debt being carried by those people is tied up in their mortgages — if they own a home at all. Contrast that with the other three clusters: the value of everyone else's debt is lower than the value of their homes.

So all that's pretty interesting, but it's different from what we did last time, right? At this point in the last lesson, we made a scatter plot. This was a straightforward task because we only worked with two features, so we could plot the data points in two dimensions. But now X has five dimensions! How can we plot this to give stakeholders a sense of our clusters?

Since we're working with a computer screen, we don't have much of a choice about the number of dimensions we can use: it's got to be two. So, if we're going to do anything like the scatter plot we made before, we'll need to take our 5-dimensional data and change it into something we can look at in 2 dimensions.

Task 6.3.19: Create a PCA transformer, use it to reduce the dimensionality of the data in X to 2, and then put the transformed data into a DataFrame named X_pca. The columns of X_pca should be named "PC1" and "PC2".

In [30]:
# Instantiate transformer
pca = PCA(n_components=2, random_state=42)

# Transform `X`
X_t = pca.fit_transform(X)

# Put `X_t` into DataFrame
X_pca = pd.DataFrame(X_t, columns=["PC1", "PC2"])

# print("X_pca type:", type(X_pca))
# print("X_pca shape:", X_pca.shape)
X_pca.head()


,PC1,PC2
0,-249467.153516,-20504.439053
1,-245134.044844,-21621.580659
2,-247019.246103,-17108.088264
3,-238646.799054,-22031.877446
4,-242436.415674,-18742.103863


In [31]:
X_t

array([[-249467.15351602,  -20504.43905256],
       [-245134.04484418,  -21621.58065865],
       [-247019.24610329,  -17108.08826415],
       ...,
       [ 377761.14569004, -232565.13594104],
       [ 383102.56908824, -234632.88202809],
       [ 390285.86263244, -237413.64401819]])

So there we go: our five dimensions have been reduced to two. Let's make a scatter plot and see what we get.

Task 6.3.20: Use plotly express to create a scatter plot of X_pca using seaborn. Be sure to color the data points using the labels generated by your final_model. Label the x-axis "PC1", the y-axis "PC2", and use the title "PCA Representation of Clusters".

In [32]:
labels.astype(str)

array(['0', '0', '0', ..., '1', '1', '1'], dtype='<U11')

In [33]:
# Create scatter plot of `PC2` vs `PC1`
fig = px.scatter(
    data_frame=X_pca,
    x="PC1",
    y="PC2",
    color=labels.astype(str),
    title="PCA Representation of Clusters"
)
fig.update_layout(xaxis_title="PC1", yaxis_title="PC2")

fig.show()

/opt/conda/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



One limitation of this plot is that it's hard to explain what the axes here represent. In fact, both of them are a combination of the five features we originally had in X, which means this is pretty abstract. Still, it's the best way we have to show as much information as possible as an explanatory tool for people outside the data science community.

So what does this graph mean? It means that we made four tightly-grouped clusters that share some key features. If we were presenting this to a group of stakeholders, it might be useful to show this graph first as a kind of warm-up, since most people understand how a two-dimensional object works. Then we could move on to a more nuanced analysis of the data.

Just something to keep in mind as you continue your data science journey.